In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Do not forget so space out the requests
spotify = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials())

# 1 - Build a population of artists
artist_ids = [
    "11E9GHIAzJRKuECEUSEuqh", # Cheb Bello 
    "3e3cKwH1kUr02bvIm7VaIe", # Kader Japonais
    "5CZ5sGdn0X47HhndSYKqdz", # Cheba Dalila
    "0c3dDCJfxcT4lYNugbKvJt", # Warda
    "4iCrZzxACYPYcoS71DgjWW", # Bilal Sghir
    "59N7N5tX53jyPhAmsRi4or", # Cheb Bilal
    "4ZzMtjQsjtaAOm3GPqmjeQ" # Cheb Djalil 
]

artist_map = {
    "11E9GHIAzJRKuECEUSEuqh": "Cheb Bello",  
    "3e3cKwH1kUr02bvIm7VaIe": "Kader Japonais",
    "5CZ5sGdn0X47HhndSYKqdz": "Cheba Dalila",
    "0c3dDCJfxcT4lYNugbKvJt": "Warda",
    "4iCrZzxACYPYcoS71DgjWW": "Bilal Sghir",
    "59N7N5tX53jyPhAmsRi4or": "Cheb Bilal",
    "4ZzMtjQsjtaAOm3GPqmjeQ" : "Cheb Djalil"
}

# 1.5 - Get basic artist info (Monthly listeners for instance)
selected_artists = spotify.artists(artist_ids).get("artists")

# 2 - Get their top tracks
top_tracks = list()
for artist_id in artist_ids:
    artist_top_tracks = spotify.artist_top_tracks(artist_id).get("tracks")
    for artist_top_track in artist_top_tracks: 
        artist_top_track["main_artist"] = artist_map.get(artist_id)

    top_tracks.extend(artist_top_tracks)

# 3 - Get maximum track features
def get_track_id(track):
    return track.get("id")

tracks_ids = list(map(get_track_id, top_tracks))
audio_features = spotify.audio_features(tracks=tracks_ids)

# 4 - Store the data in a CSV file
# 5 - Create a notebook and explore the data

In [22]:
import pandas as pd 
import numpy as np

af = pd.DataFrame(audio_features)
tt = pd.DataFrame(top_tracks)

df = pd.merge(tt, af, how="left", on="id")
df = df[['id', 'name', 'main_artist', 'duration_ms_x', 'explicit', 'danceability', 'loudness', 'energy', 'instrumentalness', 'liveness', 'valence', 'speechiness', 'tempo', 'popularity']]
df = df.sort_values(by=['popularity'], ascending=False)

df.head()

,id,name,main_artist,duration_ms_x,explicit,danceability,loudness,energy,instrumentalness,liveness,valence,speechiness,tempo,popularity
10,1Ld7ou1JXz77rg70AG7usC,El Walida,Kader Japonais,252000,False,0.611,-8.611,0.743,0.000000,0.1110,0.541,0.0833,79.996,56
0,1mhKtcpIVEwvE7qs32HcwI,Manich Kima Bakri,Cheb Bello,345652,False,0.705,-4.308,0.894,0.000005,0.0725,0.890,0.0950,96.066,47
1,1Z3UznHCGkclMLMOBRxeHN,3adyani Bel Gros - Remix Master T,Cheb Bello,226811,False,0.672,-6.673,0.787,0.543000,0.2060,0.594,0.0452,91.992,47
51,4KuO9X4amlOVQwsU2DRONy,Di Ayza Kalem,Cheb Bilal,351173,False,0.452,-7.722,0.881,0.005230,0.0453,0.970,0.0895,110.371,45
50,4dWR4PFw9WbkUrTmLmdeDB,Bienvenue chez les bylkas,Cheb Bilal,242533,False,0.627,-4.176,0.869,0.000000,0.6610,0.885,0.2270,98.662,45


In [24]:
df.to_csv("out.csv", index=False)